In [1]:
import torch
import numpy as np
import pandas as pd
from datasets import DatasetDict, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, AutoConfig
from sklearn.metrics import f1_score, accuracy_score

2025-09-20 14:55:25.088064: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset = load_dataset('liar', trust_remote_code=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1284
    })
})

In [3]:
model_path_name = 'distilbert/distilbert-base-uncased'

In [4]:
# 'false' (0), 'half-true' (1), 'mostly-true' (2), 'true' (3), 'barely-true' (4), 'pants-fire' (5)
idx_labels = {0: 'false', 1: 'half-true', 2: 'mostly-true', 3: 'true', 4: 'barely-true', 5: 'pants-fire'}
labels_idx = {'false': 0, 'half-true': 1, 'mostly-true': 2, 'true': 3, 'barely-true': 4, 'pants-fire': 5}

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(model_path_name,
                                                          num_labels=len(idx_labels),
                                                          id2label=idx_labels,
                                                          label2id=labels_idx)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# freezing of distilbert parameters
for p in model.distilbert.parameters():
    p.requires_grad = False
# unfreeze last two layers
for p in model.distilbert.transformer.layer[-2:].parameters():
    p.requires_grad = True

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path_name)

In [8]:
def preprocess_func(data):
    return tokenizer(data['statement'], truncation=True, max_length = 128, padding='max_length')

In [9]:
tokenized_data = dataset.map(preprocess_func, batched=True)
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# keep the 'necessary' columns (label, input_ids, and the mask), cleaner
features = ['id', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']
tokenized_data = tokenized_data.remove_columns(features)

tokenized_data

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1284
    })
})

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred    
    logits = torch.tensor(logits)
    probs = torch.nn.functional.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)    
    return {
        'accuracy:': accuracy_score(labels, preds),
        'macro_f1 score:': f1_score(labels, preds, average='macro')
    }    

In [12]:
config = AutoConfig.from_pretrained(
    model_path_name,
    num_labels=6,
    hidden_dropout_prob=0.2
)

In [17]:
lr = 2e-5
batch_size = 16
num_epochs = 5

training_args = TrainingArguments(
    output_dir='../output/distilbert-liar-classifier',
    run_name='distilbert-liar-classif-run',
    learning_rate=lr, 
    lr_scheduler_type='linear',
    warmup_ratio=0.1,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_macro_f1 score:'    
)

In [19]:
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy:,Macro F1 score:
1,1.668600,1.702815,0.247664,0.216278
2,1.647400,1.683810,0.264019,0.236016
3,1.600900,1.682716,0.262461,0.244338
4,1.558200,1.692566,0.258567,0.237617
5,1.527600,1.679360,0.281153,0.270948


TrainOutput(global_step=3210, training_loss=1.6005373185297411, metrics={'train_runtime': 1786.4663, 'train_samples_per_second': 28.741, 'train_steps_per_second': 1.797, 'total_flos': 1700505941783040.0, 'train_loss': 1.6005373185297411, 'epoch': 5.0})